# Python 数据模型

## 1.1 一摞Python风格的纸牌

In [18]:
import collections

# 命名元组: 和C中的结构体很相似
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [19]:
berrd_card = Card('7', 'dimonds')
berrd_card

Card(rank='7', suit='dimonds')

In [20]:
deck = FrenchDeck()
len(deck)   # 使用len函数查看一叠牌有多少张

52

### []重载

In [21]:
deck[0], deck[-1] # __getitem__ 重载函数, 类似于C中的[]重载

(Card(rank='2', suit='spades'), Card(rank='A', suit='hearts'))

> 随机抽取一张纸牌

In [22]:
from random import choice
choice(deck), choice(deck), choice(deck)

(Card(rank='8', suit='spades'),
 Card(rank='4', suit='clubs'),
 Card(rank='6', suit='diamonds'))

* 作为类的用户, 不必记住各类标准操作的各式名称
* 可以更加方便地使用Python标准库, 比如random.choice 函数, 从而不用重新发明轮子

> 因为 __getitem__ 方法把[]操作交给了 self._cards 列表, 所以我们的deck类自动支持切片操作。

In [23]:
deck[-3:]

[Card(rank='Q', suit='hearts'),
 Card(rank='K', suit='hearts'),
 Card(rank='A', suit='hearts')]

### 迭代

> 同时也可以进行迭代

In [24]:
for card in deck:
    if card.rank == '2':
        print(card)

Card(rank='2', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='2', suit='hearts')


> 反向迭代也可以

In [25]:
for card in reversed(deck): # doctest: +ELLIPSIS
    print(card)
    if card.rank == 'J':
        break

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')


> 迭代通常是隐式的。in 运算符也直接可以用在我们的FrenchDeck类上

In [26]:
Card('Q', 'hearts') in deck, Card("1", 'clubs') in deck

(True, False)

### 排序

> 通常, 按照常规, 我们用点数来判定大小, 2最小, A最大。同时对于花色 依次是 黑桃、红桃、方块、梅花

In [27]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [28]:
# 排序后
cards = [card for card in  sorted(deck, key=spades_high)]
cards

[Card(rank='2', suit='clubs'),
 Card(rank='2', suit='diamonds'),
 Card(rank='2', suit='hearts'),
 Card(rank='2', suit='spades'),
 Card(rank='3', suit='clubs'),
 Card(rank='3', suit='diamonds'),
 Card(rank='3', suit='hearts'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='clubs'),
 Card(rank='4', suit='diamonds'),
 Card(rank='4', suit='hearts'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='clubs'),
 Card(rank='5', suit='diamonds'),
 Card(rank='5', suit='hearts'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='clubs'),
 Card(rank='6', suit='diamonds'),
 Card(rank='6', suit='hearts'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='clubs'),
 Card(rank='7', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='clubs'),
 Card(rank='8', suit='diamonds'),
 Card(rank='8', suit='hearts'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='clubs'),
 Card(rank='9', suit='diamonds'),
 Card(rank='9', suit='hearts'),


### ?? 如何洗牌

> 按照目前的设计, FrencnDeck 不能洗牌, 因为这摞牌是不可变的。在第11章会讲到, 只需要一行代码实现 __setitem__ , 洗牌功能就不是问题了

## 1.2 如何使用特殊方法

特殊方法的存在是为了被Python解释器调用的, 你自己并不需要调用它们。也就是说没有my_object.__len__() 的 写法, 而应该使用len(my_object)。 在执行 len(my_object) 的时候, 如果my_object是自定义类对象, 那么Python 会自己调用其中由你自己实现的 __len__ 方法

### 1.2.1 模拟数值类型

In [29]:
import math

class Vector:

    def __init__(self, *args):
        self._vector = list(args)

    def __repr__(self):
        """打印"""
        return f"Vector{[i for i in self._vector]}".replace("[", "(").replace("]", ")")

    def __len__(self):
        """长度"""
        return len(self._vector)
    
    def __getitem__(self, pos):
        """[]重载"""
        return self._vector[pos]
    
    def __abs__(self):
        """求模"""
        return math.sqrt(sum((i**2 for i in self)))

    def __add__(self, other):
        """加法"""
        if not isinstance(other, Vector):
            raise TypeError(f"{other} type not is Vector!")
        if len(self) != len(other):
            raise ValueError(f"len({other}) !=  len({self})")
        return Vector(*[x + y for x, y in zip(self, other)])
    
    def __mul__(self, other):
        """乘法"""
        if isinstance(other, Vector):
            if len(self) != len(other):
                raise ValueError(f"len({other}) !=  len({self})")
            return Vector(*[x * y for x, y in zip(self, other)])
        elif isinstance(other, (float, int)):
            return Vector(*[other * x for x in self])
        else:
            raise TypeError(f"{other} type not is [Vector、float、int]")
        
    def __rmul__(self, other):
        return self.__mul__(other)

In [30]:
x1 = Vector(12, 442, 45, 5636, 6)
x2 = Vector(13, 45, 22, -10, 23)
x3 = Vector(3, 4)
x4 = [2, 4]
print(x1, x2, x3, x4)

Vector(12, 442, 45, 5636, 6) Vector(13, 45, 22, -10, 23) Vector(3, 4) [2, 4]


#### 求向量的模

In [31]:
abs(x3)

5.0

#### 相加

In [32]:
x1 + x2

Vector(25, 487, 67, 5626, 29)

In [34]:
try:
    x1 + x3
except Exception as e:
    print(e)

len(Vector(3, 4)) !=  len(Vector(12, 442, 45, 5636, 6))


#### 向量乘法

In [35]:
x1 * 2, x1 * 0.5,  x1 * x2

(Vector(24, 884, 90, 11272, 12),
 Vector(6.0, 221.0, 22.5, 2818.0, 3.0),
 Vector(156, 19890, 990, -56360, 138))

In [36]:
2 * x1

Vector(24, 884, 90, 11272, 12)

In [38]:
try:
    x1 * x3
except Exception as e:
    print(e)

len(Vector(3, 4)) !=  len(Vector(12, 442, 45, 5636, 6))


> 虽然代码中有大量的特殊方法, 但是代码中很少直接调用, 一般只有Python的解释器会频繁调用它们

### 1.2.1 字符串表示形式

* __repr__ 和 __str__ 的 区别在于, 后者是在 str 函数被使用, 或者print 打印时使用。 如果想要实现者两个方法中的一个, __repr__ 是更好的选择

### 1.2.1 算数运算符

* 通过 __add__ 和 __mul__ 实现了相加和相乘,  乘法的交换律则通过 __rmul__ 实现

### 1.2.4 自定义的布尔值

* 略

## 1.3 特殊方法一览

> 详见书

## 1.4 为什么len不是普通方法

> 对于内置类型， len(x) 的速度非常快, 背后的原因是CPython会直接从C结构体中读取对象长度, 完全不会调用任何方法。

## 1.5 本章小结

<img src="./images/第一章总结.jpg" width="70%">